In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def fetch_mnist():
    mnist = fetch_openml('mnist_784' , version=1 , as_frame=False)
    X , y = mnist.data , mnist.target.astype(int)

    mask = (y == 0) | (y == 1)
    X_filtered , y_filtered = X[mask] , y[mask]

    shuffle_index = np.random.permutation(len(X_filtered))

    X_shuffled = X_filtered[shuffle_index]
    y_shuffled = y_filtered[shuffle_index]

    return X_shuffled , y_shuffled

def preprocess_image(x , n_components):
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)

    x_scaled = scaler.fit_transform(x)
    x_pca = pca.fit_transform(x_scaled)

    x_pca_normalized = 2.0*np.pi*(x_pca - x_pca.min(axis=0)) / (x_pca.max(axis=0) - x_pca.min(axis=0))
    return x_pca_normalized

In [ ]:
from pennylane import numpy as np
import pennylane as qml

def create_qnn(n_layers , n_qubits):
    dev = qml.device("default.qubit")
    @qml.qnode(dev)

    def circuit(inputs , params):
        
        # Angle Encoding
        for q , feature in enumerate(inputs):
            qml.RX(feature , wires=q)

        # Setup Circuit
        for layer in range(n_layers):

            for qubit in range(n_qubits):
                qml.RX(params[layer][qubit][0] , wires=qubit)
                qml.RY(params[layer][qubit][1] , wires=qubit)
                qml.RZ(params[layer][qubit][2] , wires=qubit)

            for qubit in range(n_qubits):
                if qubit != n_qubits-1:
                    qml.CNOT(wires=[qubit , qubit+1])
                else:
                    qml.CNOT(wires=[qubit , 0])

                # Measure first qubit and return result
                result = qml.probs(wires=0)
                return result
            
        return circuit